<a href="https://colab.research.google.com/github/ooDoo6169/Python_Study/blob/master/cnn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.  library import

In [3]:
# -*- coding: utf-8 -*-
# 0. library setting
import warnings
warnings.simplefilter(action = 'ignore') # error가 아닌 warnning은 무시
# data import
import openpyxl as px

# pandas & numpy
import pandas as pd
import numpy as np

# vis
import seaborn as sns
import matplotlib.pyplot as plt # plot
%matplotlib inline
import plotly as py
import plotly.graph_objs as go

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# tensor
import tensorflow as tf

# keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils # for using to_categorical
from keras.layers import Flatten, Conv2D, MaxPooling2D, LSTM # CNN
from keras.callbacks import ModelCheckpoint, EarlyStopping # EarlyStopping
from keras.datasets import mnist, cifar10

# tqdm
from tqdm import trange, tqdm_notebook

import requests 
import os

# 재연성 
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

Using TensorFlow backend.


# 2. data import

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  

# from google.colab import drive
# drive.mount('/content/gdrive')

## 1) google drive 계정 연결

In [0]:
!pip install -U -q PyDrive


from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials 


# 1. Authenticate and create the PyDrive client.

auth.authenticate_user()

gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)


## 2) data load

In [0]:
file_id = '1pahiBlTvw5SRyGwLD2_IJAm3GZUwxDKC' # file_id 는 Google Drive 에서 얻어낸 것

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('train_1_1.xlsx') # GetContentFile() 에 명시한 파일명으로 Colaboratory 에 저장

## 3) import data as dataframe

In [7]:
!ls -al # Colaboratory의 현재 폴더에 들어있는 파일들 확인

total 81868
drwxr-xr-x 1 root root     4096 Feb  8 04:21 .
drwxr-xr-x 1 root root     4096 Feb  8 04:20 ..
-rw-r--r-- 1 root root     2660 Feb  8 04:21 adc.json
drwxr-xr-x 1 root root     4096 Feb  8 04:21 .config
drwxr-xr-x 1 root root     4096 Feb  6 17:31 sample_data
-rw-r--r-- 1 root root 83801373 Feb  8 04:39 train_1_1.xlsx


In [8]:
count = 2
li=[]
for i in range(1,count) :
    W = px.load_workbook('train_1_'+str(i)+'.xlsx')
    p = W['sheet1']

    for row in p.values:
        lrow = list(row)
        li.append(lrow)
    print(i)
    

1


# 3. Feature Selection

In [9]:
df = pd.DataFrame(np.array(li).reshape(
len(li),12), columns = ['bcateid', 'brand', 'dcateid', 'image','maker', 'mcateid', 'model', 'pid', 'price', 'product', 'scateid', 'updttm'])
len(df.index)

10000

In [10]:
# reorder the columns
cols =  ['product','image','bcateid','mcateid','scateid']
df = df.ix[:,cols]
df.head()

,product,image,bcateid,mcateid,scateid
0,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),0.75723773 0.8849334 0.0776659 0.26645592 0.37...,1,1,2
1,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][...,0.08962844 0.022527816 0.0 0.2439073 0.3126426...,3,3,4
2,크리비아 기모 3부 속바지 GLG4314P,1.0080119 2.61942 0.13982898 1.0908707 0.15845...,5,5,6
3,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,0.0980237 0.052997973 0.04391024 2.1106048 0.4...,7,7,8
4,코드프리혈당시험지50매/코드프리시험지/최장유효기간,0.18871741 0.07393849 0.0 0.0 0.023571586 0.01...,10,9,11


# 4. Data split

In [11]:
# change str to list
image_list = [df['image'][0].split(' ')]
for i in range(1,len(df)) :
    image_list += [df['image'][i].split(' ')]
len(image_list)

10000

In [0]:
# data split
X_train, X_test, y_train, y_test = train_test_split(image_list, df[['bcateid', 'mcateid','scateid']], random_state = 0)

In [13]:
X_train[0][0]

'0.021697935'

# 5. modeling

In [0]:
# layer
model = Sequential()
model.add(Dense(2048, activation='relu'))# fully connected
model.add(Dropout(0.5))
model.add(Dense(len(set(df['bcateid'])), activation='softmax')) # final output layer

# compile
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

# model run
hist = model.fit(X_train, y_train['bcateid'],
                 validation_data = (X_test, y_test['bcateid']),
                 epochs =15,
#                  epochs =30,
                 batch_size = 200)

Instructions for updating:
Colocations handled automatically by placer.
